In [2]:
%pip install pyradiomics dicom_numpy pydicom plotly matplotlib scikit-image simpleITK pynrrd dicom2nifti NiBabel NiLearn openpyxl pydicom-seg

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dicom_numpy
import pydicom as dicom

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import SimpleITK as sitk
import os

from radiomics import featureextractor

from tqdm import tqdm

In [4]:
metadata = pd.read_csv("../../Dataset/Duke MRI/manifest-1654812109500/metadata.csv")
annotation_boxes = pd.read_excel("../../Dataset/Annotation_Boxes.xlsx").set_index("Patient ID")

DATASET_PATH = '../../New Dataset/'
SEGMENTED_DATASET_PATH = '../../Segmented Dataset/'

In [5]:
annotation_boxes.loc['Breast_MRI_001']

Start Row       234
End Row         271
Start Column    308
End Column      341
Start Slice      89
End Slice       112
Name: Breast_MRI_001, dtype: int64

In [35]:
feat_ext = featureextractor.RadiomicsFeatureExtractor()
feat_ext.settings['minimumROIDimensions'] = 2
feat_ext.settings['normalize'] = True
feat_ext.settings['additionalInfo'] = False
feat_ext.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': True,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': None,
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': False,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': False}

In [36]:
feats = feat_ext.execute('../../New Dataset/Breast_MRI_001/11_ax_dyn_3rd_pass.nii.gz', '../../New Dataset/Breast_MRI_001/segment.nii.gz')

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [37]:
for key, value in feats.items():
    feats[key] = value.item()

In [38]:
pd.DataFrame(feats, index = [1])

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
1,0.891808,0.850437,29.186299,34.319179,38.431136,36.073737,39.309346,46.240422,19914.791496,30.606121,...,2.0,0.125,1.0,0.000071,1.971356e+08,0.591689,0.845129,0.000084,2.768233e-09,0.916062


In [39]:
features = pd.DataFrame()

for patient in tqdm(os.listdir(DATASET_PATH)):
    pat_dir = os.path.join(DATASET_PATH, patient)
    x = annotation_boxes.loc[patient]
    row1 = x['Start Row']
    row2 = x['End Row']

    col1 = x['Start Column']
    col2 = x['End Column']

    slice1 = x['Start Slice']
    slice2 = x['End Slice']
    for nifti_file in os.listdir(pat_dir):
        if(nifti_file == 'segment.nii.gz'):
            continue
        old_file_path = os.path.join(DATASET_PATH, patient)

        nifti_path = os.path.join(old_file_path, nifti_file)
        segment_path = os.path.join(old_file_path, 'segment.nii.gz')

        nifti_img = nib.load(nifti_path)
        mask = np.zeros(nifti_img.dataobj.shape)
        mask[row1:row2, col1:col2, slice1:slice2] = 1
        new_img = nib.Nifti1Image(mask, affine = nifti_img.affine)
        nib.save(new_img, segment_path)

        # feature extraction
        feats = feat_ext.execute(nifti_path, segment_path)
        for key, value in feats.items():
            feats[key] = value.item()

        ft_df = pd.DataFrame(feats, index = [0])
        ft_df['patient'] = patient
        ft_df['sequence'] = nifti_file
        features = pd.concat([features, ft_df])

100%|██████████| 922/922 [34:44<00:00,  2.26s/it]


In [40]:
features.to_csv("pyradiomics_extraction.csv", index = False)